### Problem
Consider a transport equation 
$$
2u_t+3u_x=0
$$
with boundary condition
$$
u(0,x) = f(x) = e^{-cos(x)}.
$$
The analytical solution is given by
$$
u(t,x) = f(x-\frac{3}{2}t) = e^{-cos(x-\frac{3}{2}t)}.
$$
Our goal is to approximate the solution using PINN.

In [4]:
import torch
from torch import nn, autograd
import matplotlib.pyplot as plt
import numpy as np

In [5]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
batch_size = 256

In [6]:
class PINN(nn.Module):
    def __init__(self) -> None:
        super(PINN, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(2,5),
            nn.Sigmoid(),
            nn.Linear(5,5),
            nn.Sigmoid(),
            nn.Linear(5,5),
            nn.Sigmoid(),
            nn.Linear(5,1),
            nn.Sigmoid(),
        )
    def forward(self,x,t):
        input = torch.cat([x,t],dim=1)
        output = self.net(input)
        return output

In [7]:
model = PINN().to(device)
mse = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())